In [1]:
#files used c19,c08

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [3]:
#ASSIGNING COL NAMES
column_names = ['State', 'District', 'Name', 'TRU','Edu-level','Persons.2','Males.2','Females.2','Persons.3','Males.3','Females.3']
#READING DATA
df = pd.read_excel('DDW-C19-0000.xlsx',names=column_names,skiprows=5,skipfooter=3)

In [4]:
df.columns

Index(['State', 'District', 'Name', 'TRU', 'Edu-level', 'Persons.2', 'Males.2',
       'Females.2', 'Persons.3', 'Males.3', 'Females.3'],
      dtype='object')

In [5]:
# df.head(20)

In [6]:
df.shape

(864, 11)

In [7]:
#UNIQUE LITERACY CATEGORIES 
df['Edu-level'].unique()

array(['Total', 'Illiterate', 'Literate', 'Literate but below primary',
       'Primary but below middle', 'Middle but below matric/secondary',
       'Matric/Secondary but below graduate', 'Graduate and above'],
      dtype=object)

In [8]:
# REMOVING URBAN AND RURAL ROWS and Edu-level TOTAL,LITERATE(it is removed as literacy has sub divisions)
idx = df.loc[(df['TRU']!='Total') | (df['Edu-level'] == 'Total') |(df['Edu-level'] == 'Literate')].index
df = df.drop(idx)

In [9]:
# #EXACTLY TWO LANG
# df['t_2_lang'] = df['Persons.2'] - df['Persons.3']
# df['m_2_lang'] = df['Males.2'] - df['Males.3']
# df['f_2_lang'] = df['Females.2'] - df['Females.3']


In [10]:
# df.reset_index(drop=True,inplace=True)

#### PREPROCESSING C08

In [11]:
column_names = ['table_name', 'State','District', 'Name', 'TRU','Age-group', 'Persons', 'Males', 'Females', 'Persons.1', 'Males.1', 'Females.1', 'Persons.2', 'Males.2', 'Females.2', 'Persons.3', 'Males.3', 'Females.3', 'Persons.4', 'Males.4', 'Females.4', 'Persons.5', 'Males.5', 'Females.5', 'Persons.6', 'Males.6', 'Females.6', 'Persons.7', 'Males.7', 'Females.7', 'Persons.8',
       'Males.8', 'Females.8', 'Persons.9', 'Males.9', 'Females.9', 'Persons.10', 'Males.10', 'Females.10', 'Persons.11', 'Males.11', 'Females.11', 'Persons.12', 'Males.12', 'Females.12']

In [12]:
# DDW-0000C-08.xlsx
lit_pop = pd.read_excel('DDW-0000C-08.xlsx',names=column_names,skiprows=6)

In [13]:
#Removing string "State - "
lit_pop['Name'] = lit_pop['Name'].str.replace('State - ','')

In [14]:
# lit_pop.head(10)

In [15]:
lit_pop.shape

(3132, 45)

In [16]:
#DROPPING TRU = (URBAN,RURAL) and AGE-GROUP = (All ages) as they are not needed
idx = lit_pop.loc[(lit_pop['TRU']!='Total') | (lit_pop['Age-group'] !='All ages')].index
lit_pop = lit_pop.drop(idx)

In [17]:
lit_pop.reset_index(drop=True,inplace=True)

COMBINING BASED ON GROUPINGS WE NEED

In [18]:
newdf_m = pd.DataFrame()       #MALE DATAFRAME
newdf_f = pd.DataFrame()        #FEMALE DATAFRAME

# newdf_m['Name'] = lit_pop['Name']
# newdf_f['Name'] = lit_pop['Name']

newdf_m['ill'] = lit_pop['Males.1']                        #ILLITERACY
newdf_f['ill'] = lit_pop['Females.1']

# newdf['lbp'] = lit_pop['Persons.3'] + lit_pop['Persons.4']  #LITERACY BUT BELOW PRIMARY
newdf_m['lbp_m'] = lit_pop['Persons.3']+lit_pop['Males.4']
newdf_f['lbp_f'] = lit_pop['Persons.3']+lit_pop['Females.4']

newdf_m['pbm_m'] = lit_pop['Males.5']                         #PRIMARY BUT BELOW MIDDLE
newdf_f['pbm_f'] = lit_pop['Females.5']

newdf_m['mbm_m'] = lit_pop['Males.6']                          # MIDDLE BUT BELOW MATRIC
newdf_f['mbm_f'] = lit_pop['Females.6']

#MATRIC BUT BELOW GRAD = Matric/Secondary  +     Higher secondary/Intermediate+ 
#                        Non-technical diploma   +    Technical diploma or certificate

newdf_m['mbg_m'] = lit_pop['Males.7']+lit_pop['Males.8']+lit_pop['Males.9']+lit_pop['Males.10']     #MATRIC BUT BELOW GRAD
newdf_f['mbg_f'] = lit_pop['Females.7']+lit_pop['Females.8']+lit_pop['Females.9']+lit_pop['Females.10']

newdf_m['gaa_m'] = lit_pop['Males.11']                          #GRADUATE AND ABOVE
newdf_f['gaa_f'] = lit_pop['Females.11']

In [19]:
# newdf_f

In [20]:
# COMMENT-->
# BUT THERE IS A NOTE IN C19(last line of file) which says this
#MATRIC BUT BELOW GRAD = Matric/Secondary  + Non-technical diploma   +    Technical diploma or certificate
# (i.e) no HIGER SECONDARY
# then we can use this instead of above matric but below graduate

# newdf_m['mbg_m'] = lit_pop['Males.7']+lit_pop['Males.9']+lit_pop['Males.10']     #MATRIC BUT BELOW GRAD
# newdf_f['mbg_f'] = lit_pop['Females.7']+lit_pop['Females.9']+lit_pop['Females.10']




In [21]:
# TRANSPOSING MAKES each category values AS A ROW
# and then we convert rows into columns to ease calculations
# this will result in the format required for c19 file respectively

conv_df_m = pd.melt(newdf_m.transpose())
conv_df_f = pd.melt(newdf_f.transpose())

In [22]:
# conv_df_f

In [23]:
df.shape,conv_df_m.shape,conv_df_f.shape

((216, 11), (216, 2), (216, 2))

In [24]:
df.columns

Index(['State', 'District', 'Name', 'TRU', 'Edu-level', 'Persons.2', 'Males.2',
       'Females.2', 'Persons.3', 'Males.3', 'Females.3'],
      dtype='object')

In [25]:
#EXACTLY TWO LANG
df['t_2_lang'] = df['Persons.2'] - df['Persons.3']
df['m_2_lang'] = df['Males.2'] - df['Males.3']
df['f_2_lang'] = df['Females.2'] - df['Females.3']

In [26]:
df.reset_index(drop=True,inplace=True)

CALCULATING EXACTLY TWOLANG RATIO and THREE OR MORE LANG RATIOS FOR MALE AND FEMALE

In [27]:
#twolang male/female respectively
df['tw_r_m'] = df['m_2_lang']/conv_df_m['value']
df['tw_r_f'] = df['f_2_lang']/conv_df_f['value']
#three lang male/female respectively
df['th_r_m'] = df['Males.3']/conv_df_m['value']
df['th_r_f'] = df['Females.3']/conv_df_f['value']

In [28]:
#calculating onelang ratios using twolang and three lang male/female respectively
df['one_r_m'] = 1-(df['tw_r_m'] + df['th_r_m'])
df['one_r_f'] = 1-(df['tw_r_f'] + df['th_r_f'])

In [29]:
#TO CALCULATE "iteracy group with highest ratio of population"
def the_func(df_new):
    indexes = []
    for i in range(36):     #iterating for states
        t = i*6
        indexes.append(df_new.iloc[t:t+6,-1].idxmax())   #finding index of maxratio
        
    df_new = df_new.iloc[indexes]      #taking only indexes of maxratio
    return df_new

creating SEPERATE DATAFRAMES FOR EACH CATEGORY OF LANG

In [30]:
df_one_lang_male = the_func(df[['State','Edu-level','one_r_m']])
df_one_lang_female = the_func(df[['State','Edu-level','one_r_f']])

df_one_lang_male.reset_index(inplace=True,drop=True)
df_one_lang_female.reset_index(inplace=True,drop=True)

#COMBINING MALE AND FEMALE INTOA SINGLE DATAFRAME
df_one_lang = df_one_lang_male.copy()
df_one_lang[['Edu-level-female','one_r_f']] = df_one_lang_female[['Edu-level','one_r_f']]

In [31]:
df_two_lang_male = the_func(df[['State','Edu-level','tw_r_m']])
df_two_lang_female = the_func(df[['State','Edu-level','tw_r_f']])

df_two_lang_male.reset_index(inplace=True,drop=True)
df_two_lang_female.reset_index(inplace=True,drop=True)

#COMBINING MALE AND FEMALE INTOA SINGLE DATAFRAME
df_two_lang = df_two_lang_male.copy()
df_two_lang[['Edu-level-female','tw_r_f']] = df_two_lang_female[['Edu-level','tw_r_f']]

In [32]:
df_three_lang_male = the_func(df[['State','Edu-level','th_r_m']])
df_three_lang_female = the_func(df[['State','Edu-level','th_r_f']])

df_three_lang_male.reset_index(inplace=True,drop=True)
df_three_lang_female.reset_index(inplace=True,drop=True)

#COMBINING MALE AND FEMALE INTOA SINGLE DATAFRAME
df_three_lang = df_three_lang_male.copy()
df_three_lang[['Edu-level-female','th_r_f']] = df_three_lang_female[['Edu-level','th_r_f']]

In [33]:
#CHANGING COLUMN NAMES
df_one_lang.rename(columns={'State':'state/ut','Edu-level':'literacy-group-males','one_r_m':'ratio-males','Edu-level-female':'literacy-group-females','one_r_f':'ratio-females'}, inplace=True)
df_two_lang.rename(columns={'State':'state/ut','Edu-level':'literacy-group-males','tw_r_m':'ratio-males','Edu-level-female':'literacy-group-females','tw_r_f':'ratio-females'}, inplace=True)
df_three_lang.rename(columns={'State':'state/ut','Edu-level':'literacy-group-males','th_r_m':'ratio-males','Edu-level-female':'literacy-group-females','th_r_f':'ratio-females'}, inplace=True)

In [34]:
df_two_lang.columns

Index(['state/ut', 'literacy-group-males', 'ratio-males',
       'literacy-group-females', 'ratio-females'],
      dtype='object')

In [35]:
#WRTITNG TO CSVS

df_one_lang.to_csv('literacy-gender-c.csv',index=False)
df_two_lang.to_csv('literacy-gender-b.csv',index=False)
df_three_lang.to_csv('literacy-gender-a.csv',index=False)